In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %cd .. 
import sys
sys.path.append("..")
import statnlpbook.util as util
import matplotlib

<!---
Latex Macros
-->
$$
\newcommand{\Xs}{\mathcal{X}}
\newcommand{\Ys}{\mathcal{Y}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\aligns}{\mathbf{a}}
\newcommand{\align}{a}
\newcommand{\source}{\mathbf{s}}
\newcommand{\target}{\mathbf{t}}
\newcommand{\ssource}{s}
\newcommand{\starget}{t}
\newcommand{\repr}{\mathbf{f}}
\newcommand{\repry}{\mathbf{g}}
\newcommand{\x}{\mathbf{x}}
\newcommand{\prob}{p}
\newcommand{\vocab}{V}
\newcommand{\params}{\boldsymbol{\theta}}
\newcommand{\param}{\theta}
\DeclareMathOperator{\perplexity}{PP}
\DeclareMathOperator{\argmax}{argmax}
\DeclareMathOperator{\argmin}{argmin}
\newcommand{\train}{\mathcal{D}}
\newcommand{\counts}[2]{\#_{#1}(#2) }
\newcommand{\length}[1]{\text{length}(#1) }
\newcommand{\indi}{\mathbb{I}}
$$

# Relation Extraction 
Relation extraction (RE) is the task of extracting semantic relations between arguments. Arguments can either be general concepts such as "a company" (ORG), "a person" (PER); or instances of such concepts (e.g. "Microsoft", "Bill Gates"), which are called proper names or named entitites (NEs). An example for a semantic relation would be "founder-of(PER, ORG)". Relation extraction therefore often builds on the task of [named entity recognition](/template/statnlpbook/02_methods/00_structuredprediction).

Relation extraction is relevant for many high-level NLP tasks, such as

* for question answering, where users ask questions such as "Who founded Microsoft?",
* for information retrieval, which often relies on large collections of structured information as background data, and 
* for text and data mining, where larger patterns in relations between concepts are discovered, e.g. temporal patterns about startups

<!-- TODO: Load NYT or other dataset, update link to NER -->

## Relation Extraction as Structured Prediction
We can formalise relation extraction as an instance of [structured prediction](/template/statnlpbook/02_methods/00_structuredprediction) where the input space \\(\Xs\\) are pairs of arguments \\(\Es\\) and supporting texts \\(\Ss\\) those arguments appear in. The output space \\(\Ys\\) is a set of relation labels such as \\(\Ys=\\{ \text{founder-of}, \text{employee-at}, \text{professor-at}, \text{NONE}\\} \\). The goal is to define a model \\(s_{\params}(\x,y)\\) that assigns high *scores* to the label \\(y\\) that fits the arguments and supporting text \\(\x\\), and lower scores otherwise. The model will be parametrized by \\(\params\\), and these parameters we will learn from some training set \\(\train\\) of \\((\x,y)\\) pairs. When we need to classify input  instances \\(\x\\) consisting again of pairs of arguments and supporting texts, we have to solve the maximization problem $\argmax_y s_{\params}(\x,y)$. Note that this frames relation extraction as a multi-class classification problem (Exercise: how could RE be formalised to predict multiple labels for each input instance and how would the example below have to be adapted for that?)


## Relation Extraction Example
Before we take a closer look at relation extraction methods, let us consider a concrete example.
<!-- Expand on this -->


## Pattern-Based Extraction
The simplest relation extraction model defines a set of textual patterns for each relation, e.g.
...
The scoring model \\(s_{\params}(\x,y)\\) then uses only one parameter and assignes scores to each relation label \\(y\\) proportional to the matches with the set of textual patterns.  
...
Patterns can also syntactic information, e.g. part of speech tags or dependency paths:
...
One of the shortcomings of this pattern-based approach is that the set of patterns has to be defined manually and the model does not learn new patterns.


## Bootstrapping
Bootstrapping relation extraction models still take as input a set of entity pairs and patterns, same as pattern-based relation extraction approaches, but they aim at discovering new patterns.
Algo:
<!-- Transform to Python code -->
- Input: set of relation types \\(\Ys\\), set of seed entity pairs \\(\Es\\), set of seed patterns for each relation (\Ps\\), set of sentences \\(\Xs\\)
- For each iteration
    - Patterns P*
    - Entity pairs E*
    - For each sentence:
        - if it contains a seed entity pair e:
            - add the path between the entity pairs to P* as a new pattern
        - if it contains a seed pattern p:
            - identify an entity pair in the sentence and add it to E*
    - P <- P + generalise(P*)
    - E <- E + generalise(E*)
We can examine the output of the model at each iteration
...
One of the things that is noticable is that with each iteration, the number of extractions we find increases, but they are less correct. One of the reasons is that the semantics of the pattern shifts, so we might try to find new patterns for lecturer-at, and because the instances share a similar context with student-at, the relation patterns for lecturer-at also capture aspects of the student-at relation. One way of improving this is with confidence values for each entity pair and pattern. (Exercise: implement a confidence weighting for patterns.)


## Supervised Relation Extraction
A different way of assigning a relation label to new instances is to follow the supervised learning paradigm, which we have already seen for other structured prediction tasks. In that case, the scoring model \\(s_{\params}(\x,y)\\) is estimated automatically based on training input sentences \\(\X\\) and their labels \\(\Ys\\).
For the model, we can use range of different classifiers, e.g. a logistic regression model, SVM, etc. 
Algo:
<!-- Transform to Python code -->
- Input: set of training sentences \\(\Xs\\) annotated with entity pairs \\(\Es\\) and relation types \\(\Ys\\) 
- features <- your_favourite_feature_extractor(training_sentences)
- model <- train_model(features, labels)
- predictions_test <- model(testing_sentences)

Features for the model are typically extracted from the path between two entities. Basic features are n-gram features, or they can be based on the syntactic structure of the input, i.e. the dependency path ([parsing](statnlpbook/chapters/parsing))
Note that here we assumed that entity pairs are part of the input, i.e. we assume the named entity recognition problem to be solved as part of the preprocessing of the data. In reality, named entities have to be recognised first.
(Exercise: use dependency parsing features instead of bag of n-gram features.)


## Distant Supervision
Supervised learning typically requires large amounts of hand-labelled training examples. Since it is time-consuming and expensive to manually label examples, it is desirable to find ways of automatically or semi-automatically producing more training data. We have already seen one example of this, bootstrapping.
Although bootstrapping can be useful, one of the downsides already discussed above is semantic drift due to the iterative nature of finding good entity pairs and patterns. 
An alternative approach to this is to distant supervision. Here, we still have a set of entity pairs \\(\Es\\), their relation types \\(\Ys\\) and a set of sentences \\(\Xs\\) as an input, but we do not require pre-defined patterns. Instead, a large number of such entity pairs and relations are obtained from a knowledge base, e.g.
...
These entity pairs and relations are then used to automatically label all sentences with relations if there exists an entity pair between which this relation holds according to the knowledge base. After sentences are labelled in this way, the rest of the algorithm is the same as for supervised relation extraction.
Algo:
<!-- Transform to Python code -->
- training_sentences <- Find training sentences with entity pairs
- SUPERVISED_RE()


## Universal Schema
<!-- Expand on this -->
Recall that for the pattern-based and bootstrapping approaches earlier, we were looking for simplified paths between entity pairs expressing a certain relation which we defined beforehand. This restricts the relation extraction problem to known relation types \\(\Ys\\). In order to overcome that limitation, we could have defined new relations on the spot and added them to \\(\Ys\\) by introducing new relation types for certain simplified paths between entity pairs.

The goal of universal schemas is to overcome the limitation of having to pre-define relations, but within the supervised learning paradigm. This is possible by thinking of paths between entity pairs as relation expressions themselves. Simplified paths between entity pairs and relation labels are no longer considered separately, but instead the paths between entity pairs and relations is modelled in the same space.

Classification model vs Universal Schema model

<!-- Show example -->


## Background
Jurafky, Dan & Martin, James H. (2016). Speech and Language Processing, Chapter 20: https://web.stanford.edu/~jurafsky/slp3/20.pdf